## Loading data

### Imports

In [ ]:
from datetime import datetime
import sys
import plotly.io as pio
import pandas as pd # type: ignore
import random
import xarray as xr # type: ignore
pio.renderers.default = "png"
sys.path.append('../')


### Reading the voting data into a dataframe

In [ ]:
from nqg_model.experiment import full_historical_data_run_plus_counterfactual
from nqg_model.load_data import retrieve_id_maps, retrieve_vote_data

## Simulation Originated Data
round_no = 24
folder_path = f'../data/r{round_no}/'

maps = retrieve_id_maps(folder_path)

sim_backtest_df = full_historical_data_run_plus_counterfactual(folder_path=folder_path, round_no=24)
sim_df = pd.concat([sim_backtest_df], ignore_index=True)


# sim_df.to_pickle(f"../data/r{round_no}/sim_results/results_{datetime.now()}.pkl.zip")

## Empirical Data
df = pd.read_csv(f'../data/r{round_no}/votes.csv')
df['user'] = df['user'].astype(str).map(lambda x: maps.UserID2Wallet.get(x, f'unknown-{x}'))
df['project'] = df['project'].astype(str).map(lambda x: maps.SubmissionID2Label.get(x, f'unknown-{x}'))
df.rename(columns={'user': 'user_ref', 'project': 'submission', 'vote_type': 'vote_type'}, inplace=True)

# fixing seed for reproducibility
random.seed(42)


In [ ]:
# extracting the neuron power tensor to render sim_df quickly
neuron_power_tensor_df = sim_df[['neuron_power_tensor', 'label']]
sim_df.drop(columns=['neuron_power_tensor'], inplace=True)

sim_df.head()

### Functions to process and visualize the data 

In [ ]:
from helper_functions import *

## Project Voting Summary

In [ ]:
project_voting_summary(df, round_no)

## User Engagement Overview

In [ ]:
user_engagement_overview(df, round_no)

In [ ]:
df.vote_type.value_counts()

## Trust Network Analysis
- Visualization of trust relationships and their impact on voting results.


In [ ]:
trust_network_analysis(sim_df)

## Scenario Cross Comparision



Terminology:
- Standard Voting: This refers to a voting mechanism wherein each participant possesses equal voting power, and the process does not involve Quorum Delegation.
- NQG: An acronym for Neural Quorum Governance, a system in which voting weight is determined through a combination of factors such as Trust, Expertise, and Voting History Neurons, in addition to the inclusion of Quorum Delegation.
- NG w/o QD: Stands for Neural Governance without Quorum Delegation. In this framework, voting weight is assigned based on Trust, Expertise, and Voting History Neurons; however, it excludes the aspect of Quorum Delegation.
- QD w/o NG: Denotes Quorum Delegation without Neural Governance. Under this system, each voter retains an equal vote, but the process allows for the delegation of votes through Quorum Delegation.

In [ ]:
scenario_map = {
    'backtesting': 'NQG',
    'no_QD': 'NG w/o QD',
    'no_NG': 'QD w/o NG',
    'no_NQG': '1 person, 1 vote'}

(_, counterfact_df, full_df, ranked_full_df) = get_results_counterfact_full_ranked_dfs(
    df, sim_df, scenario_map)

In [ ]:
counterfactual_heatmap_1(full_df)

In [ ]:
counterfactual_heatmap2(full_df)

In [ ]:
px.bar(full_df.reset_index().sort_values('result', ascending=False), x='index', y='result', labels={'result': 'Power', 'index': 'Submission'}).update_layout(height=600)


In [ ]:
counterfactual_heatmap_3(ranked_full_df)

In [ ]:
counterfactual_heatmap_3(ranked_full_df[['result', '1 person, 1 vote']].sort_values('result'))

## Neuron Tensor Analysis

In [ ]:
neuron_power_tensor = neuron_power_tensor_df[neuron_power_tensor_df['label']=='backtesting']
neuron_power_tensor=neuron_power_tensor['neuron_power_tensor'].values[1]

vote_decision_matrix=pd.DataFrame(sim_df['vote_decision_matrix'][1])

vote_tensor = xr.DataArray(vote_decision_matrix, dims=['project', 'user'])

# Ensure matching coordinates (This step is crucial for correct broadcasting and multiplication)
vote_tensor = vote_tensor.assign_coords(project=neuron_power_tensor.project.values, user=neuron_power_tensor.user.values)

# Broadcast 'vote_tensor' across the 'layer' and 'neuron' dimensions
vote_tensor = vote_tensor.expand_dims({'layer': neuron_power_tensor.layer, 'neuron': neuron_power_tensor.neuron}, axis=[-2, -1])

# Now multiply
resulting_tensor = neuron_power_tensor * vote_tensor

# Select layers
layer1 = resulting_tensor.sel(layer=0)
layer2 = resulting_tensor.sel(layer=1)

# Sum across neurons in layer1
layer1_sum = layer1.sum(dim='neuron')

# Product across neurons in layer2
layer2_prod = layer2.prod(dim='neuron')

# Calculate final power
final_power = abs(layer2_prod) * layer1_sum / abs(layer1_sum)
final_power=final_power.drop_vars('layer')

# Step 2: Adjusted Calculations
# NQG_wo_L1_trust_score: Assume layer1 neuron1 is 0
layer1_wo_neuron1 = layer1.copy()
layer1_wo_neuron1.loc[dict(neuron=0)] = 0  # Set neuron1 to 0

NQG_wo_L1_trust_score = (layer1_wo_neuron1.sum(dim='neuron')) * (layer2.prod(dim='neuron'))

# NQG_wo_L1_reputation_score: Assume layer1 neuron2 is 0
layer1_wo_neuron2 = layer1.copy()
layer1_wo_neuron2.loc[dict(neuron=1)] = 0  # Set neuron2 to 0
NQG_wo_L1_reputation_score = (layer1_wo_neuron2.sum(dim='neuron')) * (layer2.prod(dim='neuron'))

# NQG_wo_L2_past_round: Assume layer2 neuron1 is 1
layer2_wo_neuron1 = layer2.copy()
layer2_wo_neuron1.loc[dict(neuron=0)] = 1  # Set neuron1 to 1
NQG_wo_L2_past_round = (layer1.sum(dim='neuron')) * (layer2_wo_neuron1.prod(dim='neuron'))

# Convert these xarray operations into a new DataArray or Dataset
metrics = xr.Dataset({
    'NQG': final_power,
    'NQG_wo_L1_trust_score': NQG_wo_L1_trust_score,
    'NQG_wo_L1_reputation_score': NQG_wo_L1_reputation_score,
    'NQG_wo_L2_past_round': NQG_wo_L2_past_round
})

# Aggregate by user (summing over projects for simplicity, adjust as needed)
final_metrics = metrics.sum(dim='user')


# Creating the DataFrame
ranked_metrics = final_metrics.to_dataframe().reset_index()

# Applying the rank method to each column (excluding 'project') in descending order
ranked_df = ranked_metrics.set_index('project').rank(method='min', ascending=False)

# Sum the ranks across all columns to get a total ranking score for sorting
ranked_df['total_rank'] = ranked_df.sum(axis=1)

# Sort by the total rank and drop the auxiliary column
ranked_df = ranked_df.sort_values(by='total_rank').drop(columns='total_rank')

# Reset index to bring the 'project' column back
ranked_df = ranked_df.reset_index()


# # Applying the rank method, which defaults to ranking in descending order
# ranked_df = ranked_metrics.rank(method='min', ascending=False)
# ranked_df['project'] = ranked_metrics['project']  # Insert the project names back into the DataFrame
# ranked_df = ranked_df[['project', 'NQG', 'NQG_wo_L1_trust_score', 'NQG_wo_L1_reputation_score', 'NQG_wo_L2_past_round']]
# Plotting the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(ranked_df.set_index('project').astype(float), annot=True, cmap='viridis', fmt=".2f")
plt.title('Heatmap of Neuron Power Ranks by Projects')
plt.xlabel('Neuron counterfactuals')
plt.ylabel('Projects')
plt.show()

